<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Simulaci%C3%B3n_del_Modelo_de_Baum_para_Programas_VR_y_VI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
código para evaluar el modelo de William Baum 2025

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# --- Definición de las Funciones del Modelo de Baum ---
# Basado en el artículo "Explaining Performance on Interval and Ratio Schedules
# with a Molar View of Behavior" por William M. Baum (2025)

def feedback_vr(B, vr_value):
    """
    Calcula la tasa de PIE (R) para una tasa de actividad (B) en un programa VR.
    La tasa de reforzamiento (PIE rate) es directamente proporcional a la tasa de respuesta.
    R = B / VR
    """
    if vr_value <= 0:
        return np.inf
    return B / vr_value

def feedback_vi(B, vi_value_minutes, a=1.0):
    """
    Calcula la tasa de PIE (R) para una tasa de actividad (B) en un programa VI.
    La ecuación de retroalimentación es una hipérbola.
    R = 1 / (t + a/B), donde t es el intervalo promedio.
    Aquí se ajusta para que las unidades sean consistentes (eventos/minuto).
    """
    if B <= 0:
        return 0
    # Convertir el valor del VI de minutos a segundos para la fórmula.
    t_seconds = vi_value_minutes * 60
    # La fórmula original es R = B / (t*B + a) con unidades consistentes.
    # Si B está en resp/min, t debe estar en min/resp.
    # Aquí, t_seconds/60 es el valor del VI en minutos.
    return B / ((t_seconds / 60) * B + a)

def calculate_Vn(V, V0, K, t_vn=3.0):
    """
    Calcula el peso competitivo de las actividades no relacionadas con el PIE (VN).
    Esto se basa en las Ecuaciones 9 y 10 del artículo.
    """
    # Ecuación 10: Calcular la actividad no relacionada BN
    Bn_numerator = K - (V + V0)
    Bn_denominator = t_vn * (V + V0) + 1

    if Bn_denominator == 0 or Bn_numerator < 0:
        return np.inf  # VN es efectivamente infinito si no hay tiempo para BN

    Bn = Bn_numerator / Bn_denominator
    if Bn <= 0:
        return np.inf

    # Ecuación 9: Calcular VN a partir de BN
    Vn = 1 / (t_vn + 1 / Bn)
    return Vn

def calculate_B_from_R_star(R_star, params):
    """
    Calcula la tasa de actividad inducida (B) para una tasa de PIE requerida (R*).
    Esta función representa el "proceso del organismo" en el sistema de retroalimentación (Fig. 14).
    """
    K = params['K']
    c1 = params['c1']
    c0 = params['c0']
    S1 = params['S1']
    S0 = params['S0']

    if R_star <= 0:
        return 0

    # Inducción de los pesos competitivos V y V0 mediante funciones de potencia.
    V = c1 * (R_star ** S1)
    V0 = c0 * (R_star ** S0)

    # Calcular el peso competitivo de la actividad no relacionada.
    Vn = calculate_Vn(V, V0, K)

    denominator = V + V0 + Vn
    if denominator <= 0:
        return 0

    # Ecuación de asignación (matching law) para obtener la tasa de actividad B.
    B = (K * V) / denominator
    return B

# --- Lógica de Simulación ---

def find_equilibrium(schedule_type, schedule_value, params):
    """
    Encuentra el punto de equilibrio (B, R) para un programa y parámetros dados.
    El equilibrio ocurre cuando la tasa de PIE producida (R_feedback) es igual a la
    tasa de PIE requerida para mantener esa actividad (R_star).
    """

    # Se define una función de diferencia cuya raíz es el punto de equilibrio para B.
    def difference_function(B):
        if B < 0:
            return np.inf  # La tasa de actividad no puede ser negativa.

        # Calcular la tasa de PIE producida por el ambiente para una tasa de actividad B.
        if schedule_type == 'VR':
            R_feedback = feedback_vr(B, schedule_value)
        else:  # VI
            R_feedback = feedback_vi(B, schedule_value)

        if R_feedback <= 0:
            # Si no hay reforzamiento, la actividad debería ser cero.
            # La diferencia es -B para que la raíz sea B=0.
            return -B

        # Ahora, encontrar la tasa de PIE requerida (R_star) que induciría la tasa de actividad B.
        # Esto requiere resolver la inversa de `calculate_B_from_R_star`.
        # Se define una sub-función de diferencia para encontrar R_star.
        def r_star_diff(R_star):
            return calculate_B_from_R_star(R_star, params) - B

        # Se usa fsolve para encontrar la raíz, es decir, el R_star que produce B.
        try:
            # Una buena estimación inicial para R_star es R_feedback.
            R_star_solution = fsolve(r_star_diff, x0=R_feedback, xtol=1e-6)[0]
            if R_star_solution < 0: R_star_solution = 0
        except:
            # Si no se encuentra solución, el sistema es inestable en este punto.
            return np.inf

        # La función principal de diferencia es cero cuando R_feedback y R_star son iguales.
        return R_feedback - R_star_solution

    # Encontrar la raíz de la función de diferencia para hallar el B de equilibrio.
    # Se usa una estimación inicial de K/2 para B.
    try:
        B_equilibrium = fsolve(difference_function, x0=params['K'] / 2, xtol=1e-6)[0]
        # Si la tasa de actividad es muy baja, se considera como extinción (cero).
        if B_equilibrium < 0.1:
             B_equilibrium = 0
    except:
        B_equilibrium = 0

    # Calcular la tasa de PIE de equilibrio (R) a partir del B de equilibrio.
    if schedule_type == 'VR':
        R_equilibrium = feedback_vr(B_equilibrium, schedule_value)
    else:  # VI
        R_equilibrium = feedback_vi(B_equilibrium, schedule_value)

    if B_equilibrium == 0:
        R_equilibrium = 0

    return B_equilibrium, R_equilibrium

# --- Parámetros y Ejecución de la Simulación (de la Figura 18 del artículo) ---

params_vr = {'K': 200, 'c1': 10, 'c0': 5, 'S1': 0.85, 'S0': 0.05}
params_vi = {'K': 100, 'c1': 10, 'c0': 5, 'S1': 0.85, 'S0': 0.05}

vr_schedules = [640, 320, 160, 80, 40, 20, 10]
vi_schedules_min = [16, 8, 4, 2, 1, 0.5]

# --- Correr la simulación para cada programa ---
results_vr = []
for vr in vr_schedules:
    b_eq, r_eq = find_equilibrium('VR', vr, params_vr)
    if b_eq > 0:
        results_vr.append({'schedule': vr, 'B': b_eq, 'R': r_eq})

results_vi = []
for vi in vi_schedules_min:
    b_eq, r_eq = find_equilibrium('VI', vi, params_vi)
    if b_eq > 0:
        results_vi.append({'schedule': vi, 'B': b_eq, 'R': r_eq})

# --- Graficar Resultados ---
vr_B = [res['B'] for res in results_vr]
vr_R = [res['R'] for res in results_vr]
vi_B = [res['B'] for res in results_vi]
vi_R = [res['R'] for res in results_vi]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 14))
fig.suptitle("Simulación del Modelo de Baum para Programas VR y VI", fontsize=16)

# Gráfica 1: Coordenadas Logarítmicas (similar a la parte superior de la Fig. 18)
ax1.plot(vr_R, vr_B, 'o--', label='VR (simulado)', color='tab:blue')
ax1.plot(vi_R, vi_B, 's-', label='VI (simulado)', color='tab:orange')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_title('Gráfica en Coordenadas Logarítmicas')
ax1.set_xlabel('Tasa de PIE / minuto (log)')
ax1.set_ylabel('Actividad / minuto (log)')
ax1.grid(True, which="both", ls="--", linewidth=0.5)

# Marcar el punto de "ratio strain" (umbral) para VR 320, como en la Fig. 18
vr_320_res = next((res for res in results_vr if res['schedule'] == 320), None)
if vr_320_res:
    ax1.plot(vr_320_res['R'], vr_320_res['B'], 'x', markersize=12, color='red', label='VR 320 (Umbral)')
ax1.legend()

# Gráfica 2: Coordenadas Aritméticas (similar a la parte inferior de la Fig. 18)
ax2.plot(vr_R, vr_B, 'o--', label='VR (simulado)', color='tab:blue')
ax2.plot(vi_R, vi_B, 's-', label='VI (simulado)', color='tab:orange')
ax2.set_title('Gráfica en Coordenadas Aritméticas')
ax2.set_xlabel('Tasa de PIE / minuto')
ax2.set_ylabel('Actividad / minuto')
ax2.grid(True, ls="--", linewidth=0.5)
if vr_320_res:
    ax2.plot(vr_320_res['R'], vr_320_res['B'], 'x', markersize=12, color='red', label='VR 320 (Umbral)')
ax2.legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()